Install Necessary Libraries (Database Training Part)

In [ ]:
!pip install ultralytics
!nvidia-smi
pip install wandb
!pip install GPUtil

Set Training Configuration (Cloud Environment: Kaggle)

In [ ]:
# Build from YAML and transfer weights
model = YOLO('yolov8x.yaml').load('yolov8x.pt')  

# Training The Final Model
results = model.train(data="/kaggle/input/addbdvss/traffic_update.yaml",epochs=30, imgsz = 416, batch = 64 ,lr0=0.0001, dropout= 0.15, device = 0)

In [ ]:
from ultralytics import YOLO
import os
!yolo task=detect mode=train model=yolov8x.pt data="/kaggle/input/addbdvss/traffic_update.yaml" epochs=30 imgsz=640

Free Up Occupied GPU Storage

In [ ]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


Install Necessary Libraries (Test Part)

In [ ]:
ffrom google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics
!pip install cvzone
!pip install supervision
!pip install easyocr
!pip install PIL
!pip install --upgrade git+https://github.com/kbatsuren/wiktra/

Recognition of Vehicle's Registration Plate  (Cloud Environment: Google Colab)

In [ ]:
    # Import necessary libraries
    import os
    from ultralytics import YOLO
    import cv2
    from cvzone import putTextRect
    import numpy as np
    from cvzone import cornerRect
    import supervision as sv
    import easyocr
    import matplotlib.pyplot as plt
    from PIL import Image, ImageDraw, ImageFont
    #from wiktra.Wiktra import Transliterator
    from wiktra.Wiktra import translite as tr
    import unicodedata
    import re

    def extract_number(input_string):
      # Use regex to find all sequences of digits in the string
      numbers = re.findall(r'\d+', input_string)

      # Join all the found numbers into a single string
      number_string = ''.join(numbers)

      # Keep only the last 6 digits
      last_six_digits = number_string[-6:]

      # Separate the first two digits from the last four digits with a hyphen
      formatted_number = f"{last_six_digits[:2]}-{last_six_digits[2:]}"

      return formatted_number


    image_path = r"/content/drive/MyDrive/YoLo_Thesis/BDTS/Sample Inputs/1191.jpg"

    frame = cv2.imread(image_path)


    # Get the height, width, and number of channels of the frame
    H, W, _ = frame.shape

    # Coordinates of the rectangular portion (ROI)
    roi_x1, roi_y1, roi_x2, roi_y2 = 0, int(H/5), W, int((4.5*H)/5)  # Update these coordinates according to your ROI 3.3H/5
    cv2.rectangle(frame,(roi_x1, roi_y1),(roi_x2, roi_y2), (0,0,255), 4)


    # Specify the path of the YOLO model

   # model_path2 = os.path.join('.', r"/content/drive/MyDrive/YoLo_Thesis/BDV/best.pt")
    model_path3 = os.path.join('.', r"/content/drive/MyDrive/YoLo_Thesis/LPD4k/best.pt")


    # Load the YOLO model
    #model2 = YOLO(model_path2)
    model3 = YOLO(model_path3)

    tracker = sv.ByteTrack()
    trace_annotator = sv.TraceAnnotator()

    #Initialize the easyocr model
    reader = easyocr.Reader(['bn'])


    # Set a threshold for object detection confidence
    threshold = 0.5


    # Extract the ROI from the frame
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]




    # Perform object detection on the ROI using the YOLO model

    results3  = model3(roi)[0]
    #print(f"Inference Time: {results3.speed.get('inference')}ms")


    def add_bangla_text(image, text, position, font_path, font_size, color):
      # Convert the OpenCV image to a PIL image
      image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      draw = ImageDraw.Draw(image_pil)

      # Load the font
      font = ImageFont.truetype(font_path, font_size)

      # Draw the text
      #draw.text(position, text, font=font, fill=color)
      draw.multiline_text(position, text, font=font, fill=color, align="left")

      # Convert the PIL image back to an OpenCV image
      image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
      return image



    for result3 in results3.boxes.data.tolist() :
        x1, y1, x2, y2, score, class_id = result3



        x1+=roi_x1
        x2+=roi_x1
        y1+=roi_y1
        y2+=roi_y1

        x1=int(x1)
        x2=int(x2)
        y1=int(y1)
        y2=int(y2)



        w=x2-x1
        h=y2-y1

        ocr_results = []

        if score > threshold:

            im = frame[y1:y2,x1:x2]
            conf = 0.2

            gray = cv2.cvtColor(im , cv2.COLOR_RGB2GRAY)
            results = reader.readtext(gray)
            ocr = ""
            print(results)

            for result in results:
                # if len(results) == 1:
                #     ocr = result[1]
                #     print(ocr)
                print(result)

                bbox, text, prob = result

                # Adjust the bbox coordinates relative to the main frame
                bbox = [(int(point[0] + x1), int(point[1] + y1)) for point in bbox]


                ocr_results.append(result[1])  # Append the OCR result to the list

                #cv2.rectangle(frame, bbox[0], bbox[2], (0,255,0),2)
                if len(results) == 1 or len(result[1])>0 and result[2]> conf:
                    ocr = result[1]
                    #print(ocr)


                print(str(ocr))
                #bangla_text = str(ocr)
                #bangla_text = ocr

                bangla_text = " ".join(ocr_results)
                english_text = tr(str(bangla_text), 'ben')
                normalized_text = unicodedata.normalize('NFKD', english_text).encode('ASCII', 'ignore').decode('ASCII')
                uppercase_text = normalized_text.upper()


            # Split the string using regular expression to match whitespace or hyphen as separators
            words = re.split(r'\s|-', uppercase_text)

            # Remove any empty strings and leading/trailing whitespace from each word
            words = [word.strip() for word in words if word.strip()]
            print(words)

            target_word = ""

            for word in words:
                if word == "GO":
                    target_word="GA"
                elif word == "CO":
                    target_word="CHA"

                elif word == "GHO":
                    target_word="GHA"

                elif word == "LO":
                    target_word="LA"

                elif word == "HO":
                    target_word="HA"

                elif word == "CHO":
                    target_word="CAA"
                elif word == "CHO":
                    target_word="KA"




            print(bangla_text)
            print(str(english_text))
            print(uppercase_text)
            #print(number_string)
            number_string = extract_number(uppercase_text)
            print(f"Number string: {number_string}")
            print(f"DHAKA METRO-{target_word} {number_string}")


            position = (10,10)  # (x, y)
            # print(position)
            font_path = "/content/drive/MyDrive/YoLo_Thesis/BDTS/Sample Inputs/Siyam Rupali Regular.ttf"  # Path to a Bangla font file
            font_size = 40
            color = (0, 0, 255)  # Text color in RGB

            # Add the Bangla text to the image

            #frame = add_bangla_text(frame, bangla_text, position, font_path, font_size, color)

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 4)
            putTextRect(frame, "Registration Plate", [int(x1), int(y1) -15],
                         scale=1.5, thickness=2,colorR=(255,0,0), offset=7)

            putTextRect(frame, f'DHAKA METRO-{target_word} {number_string}', [int(x2)-100, int(y2) + 30],
                         scale=1.5, thickness=2,colorR=(0,0,255), offset=7)

                 #putTextRect(frame, str(ocr), [int(x1) + 8, int(y2) +30], scale=1.5, thickness=2,colorR=(0,0,255), offset=7)



    # detections = sv.Detections.from_ultralytics(results2)
    # detections = tracker.update_with_detections(detections)
    # bounding_box_annotator = sv.BoundingBoxAnnotator()
    # label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=2)



    # labels = [

    #     f"{tracker_id} {results2.names[class_id].upper()} {(confidence*100):.1f}%"
    #     for class_id, tracker_id, confidence
    #     in zip(detections.class_id, detections.tracker_id, detections.confidence)
    #   ]

    # annotated_image = bounding_box_annotator.annotate(
    #       scene=frame, detections=detections)
    # annotated_image = label_annotator.annotate(
    #       scene=annotated_image, detections=detections, labels=labels)
    # annotated_image = trace_annotator.annotate(
    #       scene=annotated_image, detections=detections)


    plt.figure(figsize=(10, 10))
    plt.imshow(frame)
    plt.axis('off')  # Hide axes
    plt.show()

    plt.figure(figsize=(10, 10))
    plt.imshow(im)
    plt.axis('off')  # Hide axes
    plt.show()


    output_path = r"/content/drive/MyDrive/YoLo_Thesis/BDTS/Sample Inputs/OutputImage.jpg"
    cv2.imwrite(output_path, frame)


